# Planet Imagery Earth Engine Delivery

In [ ]:
## Import the Planet API
import planet
from planet import Session, DataClient, OrdersClient

## Import the Earth Engine API
import ee

## Importing other packages for ordering and delivery
import json
import os
import pathlib
import time
import shapely
import requests
import geopandas as gpd
from google.colab import drive
import asyncio

### Step 1: Authenticate Earth Engine in Python environment

In [ ]:
## Trigger authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize(project='ee-jg3648')

## (Optional) Mount Google Drive to Colab environment (where my AOIs are stored)
drive.mount('/content/gdrive')

### Step 2: Authenticate Planet account for Orders API

In [ ]:
## Retrieve Planet Account API Key

## Set up & authenticate session
from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')

auth = Auth.from_key(api_key)

Enter your API key:··········


### Step 3: Define cloud delivery location for images

In [ ]:
## Create empty ImageCollection in project
TruckSeat_collection = ee.ImageCollection([])

## Define a cloud delivery configuration object (defining the destination to be GEE)
cloud_config = planet.order_request.google_earth_engine(
    project='ee-jg3648', collection='TruckSeat_collection')

## Define delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

### Step 4: Define data for delivery: AOIs and Image IDs

*Note: there are plenty of other ways to import your AOI - this is just one example*

In [ ]:
from shapely.geometry import Polygon

polygon_coords = [
    [285806.9500000002, 3961158.4178],
    [282885.2479999997, 3961185.7635999992],
    [282861.89329999965, 3958689.8296000008],
    [285783.5959000001, 3958662.4987000003],
    [285806.9500000002, 3961158.4178]
]

# Create a Shapely Polygon object
polygon = Polygon(polygon_coords)

# Check if the polygon is valid
if polygon.is_valid:
    # Extract the coordinates
    coordinates = list(polygon.exterior.coords)

    # Print the coordinates
    print(coordinates)
else:
    print("The polygon geometry is invalid.")

[(285806.9500000002, 3961158.4178), (282885.2479999997, 3961185.7635999992), (282861.89329999965, 3958689.8296000008), (285783.5959000001, 3958662.4987000003), (285806.9500000002, 3961158.4178)]


In [ ]:
## Create area of interest from GeoJSON file (from Google Drive)
TruckSeat_geojson = gpd.read_file("/content/gdrive/MyDrive/NAU - PhD INF/Sites/truck seatttt.geojson")

# Extract the geometry from the single feature in the geodataframe (site_geojson)
TruckSeat_geometry = TruckSeat_geojson.iloc[0]['geometry']

# Extract the coordinates from the geometry
TruckSeat_coordinates = list(TruckSeat_geometry.exterior.coords)

# Create polygon AOI using coordinates
TruckSeat_AOI = {
    "type":"Polygon",
    "coordinates":[[[285806.9500000002, 3961158.4178],
    [282885.2479999997, 3961185.7635999992],
    [282861.89329999965, 3958689.8296000008],
    [285783.5959000001, 3958662.4987000003],
    [285806.9500000002, 3961158.4178]]]
}


### Step 5: Define images to order

In [ ]:
## Create list of image IDs to order
TruckSeat_Images = ('20200712_173325_29_2271',
                     '20200814_173019_50_220b',
                     '20200915_173743_14_222f',
                     '20201014_173606_58_2271',
                     '20201112_182239_02_240a',
                     '20210315_173138_14_2460',
                     '20210415_173034_01_2439',
                     '20210515_181856_58_240c',
                     '20210618_174255_41_2259',
                     '20210714_173547_31_225a',
                     '20210819_172951_35_2448',
                     '20210916_173020_70_2458',
                     '20211015_172513_76_2464',
                     '20211115_181219_22_2254',
                     '20220315_172310_11_2453',
                     '20220415_181137_00_2402',
                     '20220514_172152_95_2420',
                     '20220616_172350_04_2440',
                     '20220715_171953_71_2463',
                     '20220814_180458_24_227b',
                     '20220917_175351_48_249d',
                     '20221014_172014_28_2432',
                     '20221115_181310_30_2413',
                     '20230316_175340_73_2488',
                     '20230416_175916_70_2470',
                     '20230515_174700_14_2251',
                     '20230614_180322_97_2438',
                     '20230716_175916_23_2488',
                     '20230813_173020_13_24c8',
                     '20230915_172641_76_24ce',
                     '20231015_173008_26_24a7',
                     '20231109_181137_99_2479')

image_ids = [image_id for image_id in TruckSeat_Images]

### Step 6: Build your order request

In [ ]:
from planet import order_request

item_ids = TruckSeat_Images

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:2223', kernel='cubic'),
    order_request.clip_tool(TruckSeat_AOI)
]
request = order_request.build_request(
    'Truck Seat Images', products=products, tools=tools
)

### Step 7: Create order request

In [ ]:
from planet import Session, OrdersClient

# an async Orders client to request order creation
async def main():
  async with Session(auth=auth) as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(request)

# async magic to remember: "async def" to create a coroutine, then "await" to make it run
await main()

ReadTimeout: 